In [ ]:
import os
os.chdir("ADL4CV/zero-shot/")

In [ ]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker

print(torch.cuda.is_available())


diffusion_wrapper = DiffusionWrapper('../text-to-video-ms-1.7b')
heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

video_features = diffusion_wrapper.extract_video_features('../rocket256.gif', "A rocket starting on Mars.")

In [ ]:
video_features.shape

In [ ]:
from diffusers import AutoencoderKL

# Transform feature maps with decoder
vae = AutoencoderKL.from_pretrained("ali-vilab/text-to-video-ms-1.7b", subfolder="vae", use_safetensor=True).to("cuda")

# Only use last 40 features
video_features = video_features[:, -2400:, :, :]
print(video_features.shape)
F, C, H, W = video_features.shape

video_features_4 = video_features.view(F, C // 4, 4, H, W)
video_features_decoded = torch.zeros(F, C // 4, 3, 256, 256).to("cpu")
for f in range(F):
    for c in range(C // 4):
        feat_4 = video_features_4[f, c, :, :, :].unsqueeze(0).float()
        with torch.no_grad():
            feat_4_decoded = vae.decode(feat_4).sample
        video_features_decoded[f, c, :, :, :] = feat_4_decoded.to("cpu")
video_features_decoded = video_features_decoded.view(F, -1, 256, 256)

torch.save(video_features_decoded, "video_features_decoded_last2400.pt")

In [ ]:
torch.save(video_features_decoded, "video_features_decoded_last1600.pt")
video_features_decoded.shape

In [ ]:
import torch
import numpy as np
from PIL import Image, ImageSequence

from diffusion_wrapper import DiffusionWrapper
from heatmap_generator import HeatmapGenerator
from zero_shot_tracker import ZeroShotTracker

heatmap_generator = HeatmapGenerator()
zero_shot_tracker = ZeroShotTracker()

video_features_decoded = torch.load("video_features_decoded_last2400.pt")

video_features_decoded = video_features_decoded.permute(0, 2, 3, 1).float().to("cpu")
heatmaps = heatmap_generator.generate(video_features_decoded, (93, 137, 0))

#heatmaps = torch.permute(heatmaps, (0, 3, 1, 2))
#heatmaps = torch.nn.functional.interpolate(heatmaps, size=256, mode="bilinear", align_corners=True) * 255
#
#heatmaps = heatmaps.squeeze().numpy()
#frames_gif = [Image.fromarray(f) for f in heatmaps]
#frames_gif[0].save("output/heatmaps.gif", save_all=True, append_images=frames_gif[1:], duration=100, loop=0)

tracks = zero_shot_tracker.track(heatmaps)

def load_frames(image: Image, mode='RGB'):
    return np.array([
        np.array(frame.convert(mode))
        for frame in ImageSequence.Iterator(image)
    ])

with Image.open('../rocket256.gif') as im:
    frames = load_frames(im)

zero_shot_tracker.place_marker_in_frames(frames, tracks)
heatmap_generator.safe_heatmap_as_gif(heatmaps)